In [1]:
from fst.io import access, read
import json

In [2]:
from os.path import isdir
from pathlib import Path
from glob import glob
group_path = '/volumes/raw/ch0/'
all_dirs = filter(isdir, glob('/groups/cosem/cosem/data/*'))
res = []
for a in all_dirs:
    n5s = list(map(str, Path(a).glob('*.n5')))
    rawdir = list(map(str, Path(a).glob('raw')))
    if len(rawdir) == 1:
        rawdir = rawdir[0]    
        if len(n5s) == 1:
            n5 = n5s[0]
        if len(n5s) > 1:
            n5 = list(map(str, Path(a).glob('*_raw.n5')))[0]
        res.append((rawdir, n5))

In [3]:
import numpy as np
zsteps = []
for r in res:
    rawfiles = glob(r[0] + '/*.dat')
    num_raw = len(rawfiles)
    try:
        json_file = glob(r[0] + '/*.json')[0]
        with open(json_file) as f:
            zstep = float(json.load(f)['Z-step (nm)'])
    except IndexError:
            zstep = 0.0    
    xystep = np.round(read(rawfiles[0]).header.__dict__['PixelSize'])
    zsteps.append((r[0], zstep, xystep, xystep))
    

In [7]:
wds = [read(r).header.__dict__['WD'] for r in sorted(rawfiles)]

In [6]:
[r[0] for r in res]

['/groups/cosem/cosem/data/hela_nz_2/raw',
 '/groups/cosem/cosem/data/Jurkat_Cell1_4x4x4nm/raw',
 '/groups/cosem/cosem/data/COS7_Cell11_8x8x8nm/raw',
 '/groups/cosem/cosem/data/Pancreas_Islets_4x4x4m/raw',
 '/groups/cosem/cosem/data/FS163v2_Cell2/raw',
 '/groups/cosem/cosem/data/U2OS_Cell4_8x8x8nm/raw',
 '/groups/cosem/cosem/data/FlyBrain_VNC_Sec16_4x4x4nm/raw',
 '/groups/cosem/cosem/data/HeLa_Cell2_4x4x4nm/raw',
 '/groups/cosem/cosem/data/U2OS_Cell6_8x8x8nm/raw',
 '/groups/cosem/cosem/data/ctl_1/raw',
 '/groups/cosem/cosem/data/Chlamydomonas_4x4x4nm/raw',
 '/groups/cosem/cosem/data/HeLa_Cell3_4x4x4nm/raw',
 '/groups/cosem/cosem/data/LoadID365_ROI4_8x8x8nm/raw',
 '/groups/cosem/cosem/data/TWalther_WT45_Cell2_4x4x4nm/raw',
 '/groups/cosem/cosem/data/HeLa_Cell25_8x8x8nm/raw',
 '/groups/cosem/cosem/data/Macrophage_FS80_Cell2_4x4x4nm/raw',
 '/groups/cosem/cosem/data/ctl_id8_1/raw',
 '/groups/cosem/cosem/data/HeLa_Cell21_8x8x8nm/raw',
 '/groups/cosem/cosem/data/LoadID365_ROI6_8x8x8nm/raw',


In [41]:
import numpy as np
from distributed import Client
zstep_estimates = []

for r in res:
    rawfiles = sorted(glob(r[0] + '/*.dat'))    
    query = lambda v: read(v).header.__dict__['WD']    
    with Client() as client:
        zsteps = client.gather(client.map(query, rawfiles))
    diffs = np.diff(zsteps)
    # take the median of the steps and convert from mm to nm and  round to nearest decimal
    # place
    zstep = np.round(np.median(diffs) * 10 ** 6,2)
    xystep = np.round(read(rawfiles[0]).header.__dict__['PixelSize'],2)
    zstep_estimates.append((r[0], zstep, xystep, xystep))


In [42]:
zstep_estimates

[('/groups/cosem/cosem/data/hela_nz_2/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/cosem/data/Jurkat_Cell1_4x4x4nm/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/cosem/data/COS7_Cell11_8x8x8nm/raw', 4.05, 8.0, 8.0),
 ('/groups/cosem/cosem/data/Pancreas_Islets_4x4x4m/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/cosem/data/FS163v2_Cell2/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/cosem/data/U2OS_Cell4_8x8x8nm/raw', 4.05, 8.0, 8.0),
 ('/groups/cosem/cosem/data/FlyBrain_VNC_Sec16_4x4x4nm/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/cosem/data/HeLa_Cell2_4x4x4nm/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/cosem/data/U2OS_Cell6_8x8x8nm/raw', 4.05, 8.0, 8.0),
 ('/groups/cosem/cosem/data/ctl_1/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/cosem/data/Chlamydomonas_4x4x4nm/raw', 0.0, 2.0, 2.0),
 ('/groups/cosem/cosem/data/HeLa_Cell3_4x4x4nm/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/cosem/data/LoadID365_ROI4_8x8x8nm/raw', 4.05, 8.0, 8.0),
 ('/groups/cosem/cosem/data/TWalther_WT45_Cell2_4x4x4nm/raw', 4.05, 4.0, 4.0),
 ('/groups/cosem/